# Modeling

## Importing modules and data

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

%matplotlib inline
sns.set_style('darkgrid')

In [4]:
np.random.seed(32)

In [5]:
df = pd.read_csv('../data/train_weather_spray_merged.csv')

In [6]:
df_dummied = pd.get_dummies(df, columns=['species'])

## Creating Validation Set, Scaling

In [7]:
df_dummied.drop(columns=['date', 'address', 'block', 'street', 'trap', 'addressnumberandstreet', 'nummosquitos', 'sunrise', 'sunset'], axis=1, inplace=True)

In [8]:
features = [col for col in df_dummied if col != 'wnvpresent']

In [9]:
df_dummied.columns

Index(['latitude', 'longitude', 'addressaccuracy', 'wnvpresent',
       'spray_nearby', 'station', 'tmax', 'tmin', 'tavg', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'tsra', 'sn', 'br', 'vcfg',
       'bcfg', 'hz', 'ra', 'dz', 'gr', 'mifg', 'sq', 'fg', 'ts', 'fg+', 'vcts',
       'fu', 'species_CULEX OTHER', 'species_CULEX PIPIENS',
       'species_CULEX PIPIENS/RESTUANS', 'species_CULEX RESTUANS'],
      dtype='object')

In [10]:
X = df_dummied[features]
y = df_dummied.wnvpresent

In [11]:
X.columns

Index(['latitude', 'longitude', 'addressaccuracy', 'spray_nearby', 'station',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'tsra', 'sn', 'br', 'vcfg', 'bcfg', 'hz', 'ra', 'dz', 'gr',
       'mifg', 'sq', 'fg', 'ts', 'fg+', 'vcts', 'fu', 'species_CULEX OTHER',
       'species_CULEX PIPIENS', 'species_CULEX PIPIENS/RESTUANS',
       'species_CULEX RESTUANS'],
      dtype='object')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

## Logistic Regression

In [17]:
logreg = LogisticRegression()
gs_logreg = GridSearchCV(logreg, param_grid = {'penalty': ['l1', 'l2'],
                                               'C': np.linspace(1, 10, 2),
                                               'class_weight':['balanced']}, scoring='roc_auc', verbose=1)

In [18]:
%%time
gs_logreg.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   18.0s finished


CPU times: user 21.5 s, sys: 133 ms, total: 21.7 s
Wall time: 20.6 s


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([ 1., 10.]), 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [19]:
gs_logreg.best_params_

{'C': 10.0, 'class_weight': 'balanced', 'penalty': 'l1'}

In [20]:
gs_logreg.score(X_train, y_train), gs_logreg.score(X_test, y_test)

(0.7590800534711324, 0.724642200118487)

In [24]:
model_file = 'logreg.pkl'
with open(f'../models/{model_file}', 'wb') as file:
    pickle.dump(gs_logreg, file)

## Decision Tree

In [ ]:
gs_dt = GridSearchCV(DecisionTreeClassifier(), param_grid={'min_samples_split' : [2, 3, 4, 5],
                                                           'min_samples_leaf': [2]},
                                               scoring='roc_auc')

In [ ]:
gs_dt.fit(X_train, y_train)

In [ ]:
gs_dt.score(X_train, y_train), gs_dt.score(X_test, y_test)

## Random Forest

In [ ]:
gs_rf = GridSearchCV(RandomForestClassifier(), param_grid={'min_samples_split': [2,3,4],
                                                           'min_samples_leaf': [2],
                                                           'class_weight': ['balanced'],},
                                                           scoring='roc_auc')

In [ ]:
gs_rf.fit(X_train, y_train)

In [ ]:
gs_rf.score(X_train, y_train), gs_rf.score(X_test, y_test)

In [ ]:
gs_rf.best_params_